In [1]:
import numpy as np
import cv2


# Histogramme de couleurs

In [2]:
cap = cv2.VideoCapture('videos/Extrait1-Cosmos_Laundromat1(340p).m4v')

# Convention opencv
# Si entier alors entre 0 et 255
# Si float entre 0 et 1

# Check if the video file was successfully loaded
if not cap.isOpened():
    print("Error loading video file")

# Loop through the video frames
while cap.isOpened():
    # Read the next frame from the video file
    ret, frame = cap.read()

    frameYuv = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)
    # Don't forget the [] around the frame
    hist = cv2.calcHist([frameYuv], [1,2], None, [256, 256], [0,255, 0, 255])

    # Log of histogram
    hist = np.log(hist)

    # Normalization of histogram
    hist = hist/ hist.max() * 255

    # Conversion for opencv
    hist = hist.astype(np.uint8)

    # Colormap because it's cool
    hist = cv2.applyColorMap(hist, cv2.COLORMAP_JET)

    # Check if there are no more frames
    if not ret:
        break

    # Display the frame
    cv2.imshow('frame', frame)
    cv2.imshow('histogramme', hist)

    # Wait for a key press
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the video file and close the window
cap.release()
cv2.destroyAllWindows()


/tmp/ipykernel_37781/4072166009.py:21: RuntimeWarning: divide by zero encountered in log
  hist = np.log(hist)


### Q1

On remarque des changements soudains lors des changements de plan alors que les scènes ont des changements progressifs de leur histogramme.

On pourrait calculer l'auto-corrélation entre l'histogramme courant et le précédent.
La valeur de cette auto-corrélation serait ensuite comparée à un seuil prédéfini et lorsque l'auto-corrélation dépasse ce seuil, on détecterait un changement de plan. 

Dans la cellule suivante se trouve un premier essai utilisant la somme des différences absolues (SAD) pour comparer les histogrammes. Cette méthode détecte les changements de plan, mais également les changements rapides de couleur.

In [3]:
cap = cv2.VideoCapture('videos/Extrait1-Cosmos_Laundromat1(340p).m4v')

# Convention opencv
# Si entier alors entre 0 et 255
# Si float entre 0 et 1

# Check if the video file was successfully loaded
if not cap.isOpened():
    print("Error loading video file")

threshold = 250000
hist = 0


# Loop through the video frames
while cap.isOpened():
    # Read the next frame from the video file
    ret, frame = cap.read()

    frameYuv = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)
    # Don't forget the [] around the frame
    prev_hist = hist
    hist = cv2.calcHist([frameYuv], [1,2], None, [256, 256], [0,255, 0, 255])
    # Log
    hist = np.log(hist)
    # Normalization
    hist = hist / hist.max() * 255
    hist = hist.astype(np.uint8)
    hist = cv2.applyColorMap(hist, cv2.COLORMAP_JET)

    # sum of Absolute Differencies to compare the current and previous histograms
    SAD = np.sum(np.abs(hist - prev_hist))

    if SAD > threshold :
        print('\a') # Fait un bip sonore
        print("Changement de plan       SAD : ", SAD, "      threshold", threshold)

    # Check if there are no more frames
    if not ret:
        break

    # Display the frame
    cv2.imshow('frame', frame)
    cv2.imshow('histogramme', hist)

    # Wait for a key press
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the video file and close the window
cap.release()
cv2.destroyAllWindows()

/tmp/ipykernel_37781/3443859590.py:25: RuntimeWarning: divide by zero encountered in log
  hist = np.log(hist)


(256, 256, 3)

Changement de plan       SAD :  8604880       threshold 250000
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)

Changement de plan       SAD :  259619       threshold 250000
(256, 256, 3)
(256, 256, 3)

Changement de plan       SAD :  281069       threshold 250000
(256, 256, 3)

Changement de plan       SAD :  281521       threshold 250000
(256, 256, 3)

Changement de plan       SAD :  258325       threshold 

En effet, il ne semble pas possible de définir une valeur de threshold fixe efficace. Ainsi, l'idée est venue de calculer une valeur de threshold en fonction des n derniers histogrammes afin de s'adapter à la scène en cours.

Le threshold est calculé comme suivant : threshold = threshold_gain*(moyenne des SAD des nb_hists derniers histogrammes)

threshold_gain permet donc de contrôler la sensibilité de détection de changement de scène.

Voici le code :

In [4]:
cap = cv2.VideoCapture('videos/Extrait1-Cosmos_Laundromat1(340p).m4v')

# Convention opencv
# Si entier alors entre 0 et 255
# Si float entre 0 et 1

# Check if the video file was successfully loaded
if not cap.isOpened():
    print("Error loading video file")

nb_hists = 5
hist_idx = 0 # Keep track of idx of hist to add the last one in the array of hists with minimal computation
threshold_gain = 4.5
hists_array = np.zeros(shape=(nb_hists, 256, 256, 3))
SAD = 0

hist_idx = 0
# Loop through the video frames
while cap.isOpened():
    # Read the next frame from the video file
    ret, frame = cap.read()

    frameYuv = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)

    # Don't forget the [] around the frame
    hist = cv2.calcHist([frameYuv], [1,2], None, [256, 256], [0,255, 0, 255])
    # Log
    hist = np.log(hist)
    # Normalization
    hist = hist / hist.max() * 255
    hist = hist.astype(np.uint8)
    hist = cv2.applyColorMap(hist, cv2.COLORMAP_JET)

    # Sum of Absolute Differencies of the current histogram
    SAD = np.sum(np.abs(hist - hists_array[hist_idx-1]))

    # init threshold to 0
    threshold = 0

    # Sum of Absolute Differencies to compare the current and previous histograms
    for i in range(1, nb_hists):
        threshold += np.sum(np.abs(hists_array[i] - hists_array[i-1]))
    # Adjustment of the threshold with threshold_gain
    threshold = threshold_gain*threshold/nb_hists

    if SAD > threshold :
        print('\a') # Fait un bip sonore
        print("Changement de plan       SAD : ", SAD, "      threshold", threshold)

    # Add the current histogram to the list for the upcoming histogram of the next iteration
    hists_array[hist_idx] = hist
    # Update the hist_idx to access the latest histogram
    hist_idx = hist_idx+1 if (hist_idx<nb_hists-1) else 0

    # Check if there are no more frames
    if not ret:
        break

    # Display the frame
    cv2.imshow('frame', frame)
    cv2.imshow('histogramme', hist)

    # Wait for a key press
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the video file and close the window
cap.release()
cv2.destroyAllWindows()

/tmp/ipykernel_37781/556272958.py:28: RuntimeWarning: divide by zero encountered in log
  hist = np.log(hist)



Changement de plan       SAD :  8604880.0       threshold 0.0


On a commencé les essais à threshold_gain = 2, ce qui a donné :
- L'amélioration la plus flagrante : les changements abondants et progressifs (bien que rapides) ne sont plus détectés. L'exemple le plus marquant est l'apparition dans le ciel de la tornade de couleur.
- Certains des éclairs de la scène sont souvent détectés comme des changements de scène. C'est logique car tous les pixels deviennent blancs donc la colorimétrie de la scène change brutalement et c'est ce que détecte le code créé.

Cependant, après quelques ajustements pour trouver le bon threshold_gain, le programme détecte tous (sauf 1) changements de plan et environ la moitié des éclairs.
Il est également fort probable que les transitions trop douces ne soient pas correctement détectées.

Si on devait passer en niveaux de gris, on pourrait utiliser un histogramme 1D et réaliser exactement le même traitement.
